In [1]:
#Import and set up model for text classification using DistilBERT
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [2]:
# Define the custom dataset class used during saving
class ManipulationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

# Load the preprocessed datasets from disk
# These were saved as full ManipulationDataset objects, so we must define the class first
train_dataset = torch.load('../data/processed/tokenized/train_dataset.pt', weights_only=False)
test_dataset = torch.load('../data/processed/tokenized/test_dataset.pt', weights_only=False)

# Print the number of samples in each dataset
print("Train dataset size:", len(train_dataset))
print("Test dataset size:", len(test_dataset))

# Print a few sample items from the train dataset to verify structure and content
print("\nSample items from train_dataset:")
for i in range(3):
    sample = train_dataset[i]
    print(f"Sample {i}:")
    for key, val in sample.items():
        # Print shape and first few values for tensors
        if isinstance(val, torch.Tensor):
            if val.dim() == 1:
                print(f"  {key}: shape={val.shape}, values={val[:10].tolist()}")
            else:
                print(f"  {key}: scalar={val.item()}")
        else:
            print(f"  {key}: {val}")
    print("-" * 40)


Train dataset size: 47671
Test dataset size: 11918

Sample items from train_dataset:
Sample 0:
  input_ids: shape=torch.Size([512]), values=[101, 1996, 2976, 3914, 2291, 2038, 15506, 2058, 2055, 1037]
  attention_mask: shape=torch.Size([512]), values=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  labels: scalar=1
----------------------------------------
Sample 1:
  input_ids: shape=torch.Size([512]), values=[101, 1030, 2142, 11079, 2001, 2081, 2197, 2251, 1012, 1045]
  attention_mask: shape=torch.Size([512]), values=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  labels: scalar=1
----------------------------------------
Sample 2:
  input_ids: shape=torch.Size([512]), values=[101, 2048, 18217, 1999, 2474, 18183, 3102, 1016, 1010, 1999]
  attention_mask: shape=torch.Size([512]), values=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  labels: scalar=0
----------------------------------------


In [3]:
# Load a pre-trained DistilBERT model for sequence classification
# This initializes the model with a classification head suitable for binary tasks (2 labels by default)


model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Print model summary to confirm it loaded correctly
# This will show the architecture and number of parameters
print("Model loaded successfully:")
print(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully:
DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dro

In [4]:


# Define a function to compute evaluation metrics for binary classification
def compute_metrics(pred):
    # Extract true labels and predicted class indices
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Compute precision, recall, F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    # Compute overall accuracy
    acc = accuracy_score(labels, preds)

    # Return metrics as a dictionary
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


In [5]:
# Configure training parameters for Hugging Face's Trainer API
training_args = TrainingArguments(
    output_dir="./results",                     # Where to save model checkpoints and outputs
    eval_strategy="epoch",                       # Run evaluation at the end of each epoch (use `eval_strategy` for this transformers version)
    save_strategy="epoch",                      # Save model checkpoint at the end of each epoch
    logging_dir="./logs",                       # Directory for TensorBoard logs
    per_device_train_batch_size=16,               # Training batch size per device
    per_device_eval_batch_size=16,                # Evaluation batch size per device
    num_train_epochs=3,                           # Total number of training epochs
    weight_decay=0.01,                            # Weight decay for regularization
    load_best_model_at_end=True,                  # Restore best model after training
    metric_for_best_model="f1",                 # Use F1 score to select best checkpoint
    logging_strategy="epoch",                   # Log metrics at the end of each epoch
    save_total_limit=2                            # Keep only the 2 most recent checkpoints
)

# Quick verification prints so you can confirm the TrainingArguments object is configured
print('training_args created successfully:')
print('  output_dir =', training_args.output_dir)
print('  device =', getattr(training_args, '_setup_devices', 'not-yet-initialized'))
# eval_strategy attribute may be named eval_strategy or evaluation_strategy depending on version
for attr in ('eval_strategy', 'evaluation_strategy', 'save_strategy', 'save_total_limit', 'logging_strategy', 'per_device_train_batch_size', 'per_device_eval_batch_size', 'num_train_epochs'):
    if hasattr(training_args, attr):
        print(f'  {attr} =', getattr(training_args, attr))


training_args created successfully:
  output_dir = ./results
  device = cuda:0
  eval_strategy = IntervalStrategy.EPOCH
  save_strategy = SaveStrategy.EPOCH
  save_total_limit = 2
  logging_strategy = IntervalStrategy.EPOCH
  per_device_train_batch_size = 16
  per_device_eval_batch_size = 16
  num_train_epochs = 3


In [6]:
# Set up the Hugging Face Trainer with the model, training configuration, datasets, and evaluation metrics
trainer = Trainer(
    model=model,                          # The model to be trained (e.g., a fine-tuned transformer)
    args=training_args,                   # Training arguments including batch size, epochs, logging, etc.
    train_dataset=train_dataset,         # Dataset used for training
    eval_dataset=test_dataset,           # Dataset used for evaluation during training
    compute_metrics=compute_metrics      # Function to compute evaluation metrics such as accuracy or F1 score
)

# Notify that training is starting
print("Starting model training...")

# Begin the training process
trainer.train()

# Notify that training has completed
print("Training complete. Model has been trained and evaluated.")

# Save the trained model's configuration and weights to the '../models/manipulation_detector_model' directory
# This allows the model to be reloaded later for inference or further training
model.save_pretrained("../models/manipulation_detector_model")

# Evaluate the model on the test dataset after training
eval_results = trainer.evaluate()

# Display evaluation metrics
print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value}")


Starting model training...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.281600,0.246636,0.883789,0.872096,0.948894,0.908876
2,0.201300,0.263854,0.886055,0.875365,0.948482,0.910458
3,0.133600,0.369850,0.882950,0.895430,0.915236,0.905225


Training complete. Model has been trained and evaluated.


Evaluation Results:
eval_loss: 0.263854056596756
eval_accuracy: 0.8860547071656318
eval_precision: 0.8753645239000888
eval_recall: 0.948481934331639
eval_f1: 0.9104576025319794
eval_runtime: 2115.8831
eval_samples_per_second: 5.633
eval_steps_per_second: 0.352
epoch: 3.0
